In [1]:
import os
os.chdir("C:\\Users\\hoped\\Desktop\\6.793\\project\\vascular-llm-qa")
print(os.getcwd())
mcq_fname = "C:\\Users\\hoped\\Desktop\\6.793\\project\\vascular-llm-qa\\data\\mcq.csv"


C:\Users\hoped\Desktop\6.793\project\vascular-llm-qa


In [2]:
import pandas as pd
import numpy as np
import pickle
import csv
import os
import time

In [11]:
questions_df = pd.read_csv(mcq_fname, encoding='utf-8')
questions_df

,question_id,source,labels,topic,question,answer,explanation
0,1a,apdvs,"cs, lq",carotid disorders,A 75-year-old male smoke presents with recent ...,D,Right carotid endarterectomy. The patient has ...
1,1b,apdvs,"cs, sq",carotid disorders,A 75-year-old man had temporary vision loss in...,D,Right carotid endarterectomy. The patient has ...
2,2a,apdvs,"gk, lq",carotid disorders,What is the first muscle layer encountered in...,C,The platysma is the most superficial muscle in...
3,2b,apdvs,"gk, sq",carotid disorders,"During a carotid endarterectomy, which neck mu...",C,The platysma is the most superficial muscle in...
4,3a,apdvs,"gk, lq",lower extremity arterial disease,A 65-year old male with a history of hyperten...,B,Claudication is defined as reproducible pain o...
...,...,...,...,...,...,...,...
446,56.ten,case_studies,cs,venuous disease,A 19 year old female was brought in to the Eme...,C,NaN
447,57.2,case_studies,gk,venuous disease,Which of the following statements regarding th...,"B, C, E",NaN
448,57.3,case_studies,gk,venuous disease,Which of the following statements regarding th...,"A, B, D",NaN
449,57.4,case_studies,gk,venuous disease,What are the long-term results of derivative a...,"A, C",NaN


In [19]:
# do data conversion
import json
import re
from utils.data_utils import extract_answer_choices
#from retriever.retrieval import retrieve_docs
from tqdm import tqdm

def convert_df_to_json(df):
    id_question = {}
    for _, row in tqdm(df.iterrows()):
        full_question_text = row["question"]
        try:
            question_stem, answer_choices = extract_answer_choices(full_question_text)
        except Exception as e:
            print(f"Error parsing choices for question: {full_question_text}")
            raise e

        raw_answers = str(row['answer'])
        answer_letters = re.split(r"[,\s]+", raw_answers.strip())
        letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        for a in answer_letters:
          i = letters.index(a)
          try:
            answer_choices[i]
          except Exception as e:
            print(a, i)
            print(row["question_id"])
            print("\n".join(answer_choices))
            raise e

          # make sure that all answers exist
        full_answers = [f'{letters[i]}. {x}' for i,x in enumerate(answer_choices) if letters[i] in answer_letters]

        print(question_stem)
        relevant_docs = retrieve_docs(question_stem)
        print(relevant_docs)

        example = {
            "question_id": row["question_id"],
            "question": question_stem,
            "choices": answer_choices,
            "correct_answer": answer_letters,
            "full_answers": full_answers,
            "explanation": row.get("explanation", ""),
            "source": row.get("source", ""),
            "tags": row.get("labels", "").split(","),
            "topic": row.get("topic", ""),
            "context": relevant_docs,
        }
        id_question[row["question_id"]] = example

    return id_question

id_question = convert_df_to_json(questions_df)

id_question["5a"]

1it [00:00,  7.35it/s]

A 75-year-old male smoke presents with recent visual changes to his right eye that occurred yesterday. The patient reports a shading of his visual field that resulted in momentary monocular blindness followed by return to normal vision. He has a carotid duplex showing >50% diameter reduction to his right internal carotid artery and >80% stenosis to his left internal carotid artery. What treatment should be offered to this patient?
[np.int64(1137), np.int64(2925), np.int64(1354), np.int64(198)]
1137
2925
1354
198
ocular blindness associated with
carotid-artery stenosis. N Engl J Med 2001;345:1084-90.
21. Qureshi AI, Alexandrov AV, Tegeler CH, Hobson RW 2nd, Dennis
Baker J, Hopkins LN, et al. Guidelines for screening of extracranial
carotid artery disease: a statement for healthcare professionals from the
multidisciplinary practice guidelines committee of the American Society of Neuroimaging: cosponsored by the Society of Vascular and
Interventional Neurology. J Neuroimaging 2007;17:19-4

1it [00:00,  4.27it/s]

[np.int64(1137), np.int64(117), np.int64(1354), np.int64(198)]
1137
117


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 129: character maps to <undefined>

In [ ]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json
import random

# Load FAISS index and corpus (assumes pre-built)
faiss_index = faiss.read_index("retriever/faiss_index/corpus.index")
with open("retriever/faiss_index/metadata.json") as f:
    metadata = json.load(f)

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def get_chunk_by_index(idx, chunk_dir="retriever/corpus"):
    """Given a FAISS index, load the corresponding chunk text."""
    filename = metadata[idx]["filename"]
    path = os.path.join(chunk_dir, filename)
    with open(path, "r") as f:
        return f.read()

# next; include golden for training
def retrieve_docs(question, include_relevant=True, top_k=4):
    """Retrieve documents for a question.

    If include_relevant=True: return top-k FAISS retrieved docs.
    If include_relevant=False: return true distractors, sampled randomly while excluding top matches.
    """

    # Embed the question
    question_embedding = embedder.encode([question])
    question_embedding = np.array(question_embedding).reshape(1, -1)

    # Search FAISS index
    _, indices = faiss_index.search(question_embedding, top_k)
    retrieved_indices = list(indices[0])

    if include_relevant:
        # Return the top-k most similar documents
        print(retrieved_indices[:top_k])
        chunks = []
        for idx in retrieved_indices[:top_k]:
            print(idx)
            chunk = get_chunk_by_index(idx)
            print(chunk)
            chunks.append(chunk)

        return chunk

    else:
        # Build candidate pool by excluding top-k retrieved
        all_indices = set(range(len(metadata)))
        blocked_indices = set(retrieved_indices[:top_k])
        candidate_indices = list(all_indices - blocked_indices)

        # Randomly sample true distractors
        distractor_indices = random.sample(candidate_indices, top_k)
        return [get_chunk_by_index(idx) for idx in distractor_indices]





In [39]:
models =  ["FreedomIntelligence/Apollo-0.5B", "johnsnowlabs/JSL-MedPhi2-2.7B", "skumar9/Llama-medx_v3.2", "microsoft/Phi-4-mini-instruct"]
model_name = models[3]

In [40]:
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

c:\Users\hoped\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hoped\.cache\huggingface\hub\models--microsoft--Phi-4-mini-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/3.91M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

In [ ]:
from retriever.retrieval import retrieve_docs

# Test on a new MCQ prompt
chunks = retrieve_docs("What is the primary artery supplying blood to the brain?")
text =  "\n".join([f"Document {i+1}: {chunk}" for i, chunk in enumerate(chunks)])
prompt = f"""Context:{text}
Question: What is the primary artery supplying blood to the brain?
Options:
A. Femoral artery
B. Carotid artery
C. Subclavian artery
D. Brachial artery
Answer:"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Question: What is the primary artery supplying blood to the brain?
Choices:
A. Femoral artery
B. Carotid artery
C. Subclavian artery
D. Brachial artery
Answer: ['B'] - [explanation] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs] - racs]
